In [19]:
import numpy as np
from keras.datasets import mnist  # 手書き文字認識のデータセットが入ってます。

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
import matplotlib.pyplot as plt  # 画像を描画するツール
% matplotlib inline

In [6]:
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

In [26]:
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

In [7]:
x_train.shape

(60000, 28, 28, 1)

In [27]:
y_train.shape

(60000, 10)

In [15]:
input_shape = (28, 28, 1)
num_classes = 10

In [22]:
x_in = Input(shape=input_shape, name='input')
x = Conv2D(32, (3, 3), activation='relu')(x_in)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x_in)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
prediction = Dense(num_classes, activation='relu')(x)

In [23]:
model = Model(x_in, prediction)

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               692352    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')

In [31]:
history = model.fit(x_train, y_train, epochs=10, batch_size=16, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
22656/60000 [==========>...................] - ETA: 38s - loss: nan - acc: 0.1215

KeyboardInterrupt: 